## How audio fingerprinting works

Audio fingerprinting is a method to "fingerprint", i.e., assing a unique identifier to an audio signal. The "fingerprint" can then be used later to identify or match signals.

To demonstrate audio fingerprinting, we have written a class called AudioFP that contains all the functions needed to fingerprint. We will go through the process of audio fingerprinting step-by-step by calling many of these functions. If you are interested in looking at how the functions are implemented then open the AudioFP.py file. Below, we import the AudioFP class (i.e. the AudioFP.py file) as "afp" along with other relevant packages.

In [ ]:
# Import relevant packages

from bokeh.io import output_notebook
import warnings
import AudioFP as afp

warnings.filterwarnings('ignore')
output_notebook()

### Create AudioFP object

We start by creating an object of the AudioFP class.

We can specify whether to go through the audio fingerprinting process manually (step-by-step) or automatically by entering `m` or `a` for the `process` argugment. If you leave the `process` argument blank, when the following code is executed, you will be asked to chose whether you would like to proceed manually (i.e. manually call the functions in the right order) or automatically. For this demonstration choose manually, i.e., enter `m`. By doing so, we create an empty object of the AudioFP class called `song1`. In the `AudioFP` implementation, the object has three properties, namely, the name of the song, the framerate of its signal, and lastly its fingerprint. All three of these properties will be blank to begin with. 

In [ ]:
# Create AudioFP object for a song

song1 = afp.AudioFP(process='m')

### Read audio signal

The next step is to read the signal of an audio file. Note that only `.mp3` files can be properly read with this code.

We can either read an audio file saved on the computer, or use a microphone to record an audio. The function to read from file is called `read_audiofile` while the one for recording audio is called `record_audiofile`. In both of these functions, the first input argument is the `AudioFP` object we just created (i.e. `song1`). The second argument is a boolean (`True` or `False`) for displaying plots. When the boolean argument is `True`, running the function will display any resulting plots and vice versa. The last argument is the name for the file. When reading a file this is the filename without its extension, when recording this is the name to be used for the signal. Note that in the current implementation, only `.mp3` files can be read. The outputs from both of these functions (`read_audiofile` and `record_audiofile`) are the audio signal and the framerate of the signal in that order.

In [ ]:
# Read audio signal from a file
channels, framerate = afp.AudioFP.read_audiofile(song1, True, 'vanilla_ice_ice_ice_baby')

### Create a spectrogram

Once we have the raw audio signal, we can generate a spectrogram. A [spectrogram](https://en.wikipedia.org/wiki/Spectrogram) is a visual representation of the frequency content of the signal as a function of time. The spectrogram of any audio signal can be considered unique however, it is too large to be useful as a unique fingerprint.

It can be generated by taking a Fourier transform of the signal in a short time segment (or window) and then sliding that window over the entire duration of the signal. A [Fourier transform](https://en.wikipedia.org/wiki/Fourier_transform) decomposes a signal into its constituent frequencies. For calculating a Fourier Transform numerically, a technique known as [Fast Fourier Transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform), or FFT, is used. When sliding this window over which we calculate the FFT, we can choose to have the window overlap with the previous window location. When there is no overlap, the FFT will "see" discontinuitues, resulting in spurious high-frequencies in the spectrogram. On the other hand if there is too much overlap, we are wasting computational resourses. In order to remove discontinuities, often a "window function" is multiplied to the signal within the window. The optimal degree of overlap depends on the type of window function used. In the current implementation, we use the "[Hamming](https://en.wikipedia.org/wiki/Window_function#Hamming_window)" window function for with a recommened overlap ratio is 50%. For those who are interested in digging deeper, we use the "spectrogram" function from scipy.signal for generating our spectrogram.

The function in `AudioFP` that generates a spectrogram is called `generate_spectrogram`. As described above, the two main parameters for generating the spectrogram are the number of points in the sliding segment (i.e. window length) over which to calculate the FFT and the extent of overlap between segments. In the `AudioFP` class these variables are named `nperseg` and `overlap_ratio`, respectively. The `overlap_ratio` is a fraction between 0 and 1 that multiplies the parameter `nperseg` (number of points in the sliding segment) to determine extent of overlap. Before, running the `generate_spectrogram` function, make sure to set these parameters. The first two arguments to the `generate_spectrogram` function are again, the `AudioFP` object and the boolean for displaying results. The third and fourth arguments are the raw audio signal and its framerate. This function outputs frequencies, time points, and the amplitude in that order. Note that the amplitude displayed is in the dB scale.

In [ ]:
# Generate spectrogram 
# Set parameters for generating spectrogram
afp.nperseg = 16 * 256  # window size
afp.overlap_ratio = 0.4  # degree of overlap, larger number->more overlap
f, t, sgram = afp.AudioFP.generate_spectrogram(song1, True, channels, framerate)

### Condense spectrogram data

The spectrogram of an audio signal could be considered its unique signature. Therefore to identify whether two signals are the same, one can compare their spectrograms. However, the spectrogram is essentially a quite large three dimensional (frequency, time, amplitude) array and therefore requires considerable amount of memory. To phyically store and computationally compare unique signatures in the form of spectrograms for millions of songs (Shazam has a database of several millions of songs) would be an intractable problem. So, the next step is to take all the information in the spectrogram and find a way to condense it. The way [Shazam does this](https://www.ee.columbia.edu/~dpwe/papers/Wang03-shazam.pdf) is by generating what they call a "constellation map". The constellation map is generated by finding local peaks in the spectrogram. 

Peak finding is a challenging problem in itself and for audio fingerprinting to work well, the peak finding algorithm has to be very robust. We use image processing techniques for finding local maxima. This technique involves dilating the image, finding local maxima in the dilated image, and then eroding back (For those interested in digging deeper, we use the "peak_local_max" function from the library skimage). 

In `AudioFP`, the `find_peaks` function can be used to detect local maxima in the spectrogram. We have to specify two parameters for the peak finding algorithm, the first parameter `min_peak_sep` denotes the minumum number of pixels separating two peaks, and the second `min_peak_amp` is the threshold for a maxima to be considered a peak. We have to set these two variables before calling the function `find_peaks`. The input arguments for this function are the following in that order. The `AudioFP` object, the boolean to display results, the frequencies, time points, and amplitude (i.e. output from `generate_spectrogram`) of the spectrogram. The `find_peaks` function outputs the frequencies, time points, and amplitudes of the local peaks that it finds. 

In [ ]:
# Find local peaks in the spectrogram
# Set parameters for peak finding
afp.min_peak_sep = 30  # larger sep -> less peaks -> less accuracy, but faster fingerprinting
afp.min_peak_amp = 15  # larger min amp -> less peaks -> less accuracy, but faster fingerprinting
fp, tp, peaks = afp.AudioFP.find_peaks(song1, True, f, t, sgram)

### Generate a hashed fingerprint

With the above constellation map, we have now condensed the data from the spectrogram of the audio signal. The next step is to take this condensed data and generate a fingerprint. Shazam uses a technique where a frequency of a local peak is paired with another local peak frequency in its vicinity and a time difference between the frequencies is calculated. So for each local peak frequency (anchor), we have a collection of nearby frequencies (targets) and their time deltas. This preserves local unique features in the spectrogram and is the information used to generate a fingerprint by passing it to a Hashing function. A Hashing or [Hash function](https://en.wikipedia.org/wiki/Hash_function) is a kind of function that takes data with a variable size and produces output data with a fixed size (called a Hash). Also, a Hashing functions will always produce the same Hash for the same input. The output of the Hash function is the audio fingerprint and allows us to compare signals that might be of different lengths. 

Thus, to generate the fingerprints, we have to first set the values of the following parameters which specify the size of the region for a local feature. The parameter `peak_connectivity` specifies the minimum number of neighboring peaks to use as targets. Parameters `peak_time_delta_min` and `peak_time_delta_max` specify the minimum and maximum temporal spacing between the anchor and target frequencies. The function `generate_fingerprint` in the AudioFP class can be used to create the fingerprint (For those who might want to dig deeper, we use the `MinHash` class from the datasketch library for generating the Hashes). The input arguments to `generate_fingerprint` are the `AudioFP` object and boolean for display as usual in that order, and also the local peak frequencies, time points, and amplitudes (i.e. outputs from `find_peaks`). Note that we don't really need the amplitudes at this point. Using just the temporal spacing between peak frequencies and not relying on the ampltitude makes the algorithm more robust to noise levels. The `generate_fingerprint` function updates the AudioFP object's fingerprint property and does not return any output. When the display boolean argument is set to `True`, a numeric export of the binary fingerprint is printed.

In [ ]:
afp.peak_connectivity = 15  # Number of neighboring peaks to use as target for each anchor
afp.peak_time_delta_min = 0  # Minimum spacing in time between peaks for anchor and target
afp.peak_time_delta_max = 200  # Maximum spacing in time between peaks for anchor and target
fp = afp.AudioFP.generate_fingerprint(song1, True, fp, tp, peaks)

### Something to note

There is one big difference between what we have done so far and how Shazam does their audio fingerprinting especially searching and storing. When generating the fingerprint, Shazam also stores the time point of each of the anchor frequencies. Thus, instead of one hashed fingerprint per audio signal, Shazam has a database entry for each signal that consists of the time point of each target frequency and the associated hash value. This has a key advantage when it comes to comparing signals. Knowing the time offset of each hash allows Shazam to use a much smaller subset of the entire audio signal for comparison with the original. However, to store and extract these many hashes efficiently requires creating a database which is outside the scope of this exercise. The steps we followed will also allow us to compare two signals where one is a smaller subset of the other however, the accuracy would be lower.